<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Diffusion_GPT2_improve_reliability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

use a pre-trained language model (like GPT-2) for generating responses and then apply a simple "diffusion" process to refine those responses by removing or modifying parts that could be misleading or incorrect.

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Function to generate a response
def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example prompt
prompt = "I have been prescribed two medications, Ab and Ba. Can I take them together?"

# Generate the initial response
initial_response = generate_response(prompt)
print("Initial Response:", initial_response)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Initial Response: I have been prescribed two medications, Ab and Ba. Can I take them together?

Yes.

Can I take Ab and Ba together?

Yes.

Can I take Ab and Ba together?

Yes.



In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the pad token to be the same as the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Function to generate a response with attention mask
def generate_response(prompt):
    # Tokenize the input prompt and generate attention mask
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Generate the output with the attention mask
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Function to refine the response using a simple diffusion process
def refine_response(response):
    # Simple rules for diffusion (for demonstration purposes)
    if "instead of" in response:
        response = response.replace("instead of", "and")
    if "but consult your doctor" not in response:
        response += " It is crucial to consult your doctor before making any changes."

    return response

# Example prompt
prompt = "I have been prescribed two medications, Ab and Ba. Can I take them together?"

# Generate the initial response
initial_response = generate_response(prompt)
print("Initial Response:", initial_response)

# Apply the diffusion process
refined_response = refine_response(initial_response)
print("Final Refined Response:", refined_response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Initial Response: I have been prescribed two medications, Ab and Ba. Can I take them together?

Yes.

Can I take Ab and Ba together?

Yes.

Can I take Ab and Ba together?

Yes.

Final Refined Response: I have been prescribed two medications, Ab and Ba. Can I take them together?

Yes.

Can I take Ab and Ba together?

Yes.

Can I take Ab and Ba together?

Yes.
 It is crucial to consult your doctor before making any changes.


define a basic "diffusion" process that refines the generated text by checking for certain keywords or phrases that could be potentially unsafe or misleading.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network for denoising (simulating the diffusion process)
class SimpleDenoisingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleDenoisingModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

# Example vocabulary (limited for simplicity)
vocab = ["<PAD>", "<START>", "<END>", "you", "should", "take", "Ab", "instead", "of", "Ba", "and", "consult", "your", "doctor", "before", "mixing", "medications"]
vocab_size = len(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Hyperparameters
embedding_dim = 16
hidden_dim = 32
learning_rate = 0.001
num_steps = 100

# Instantiate the model, loss function, and optimizer
model = SimpleDenoisingModel(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Example training data (noisy text)
training_data = [
    [word_to_idx[word] for word in ["<START>", "you", "should", "take", "Ab", "instead", "of", "Ba", "<END>"]],
    [word_to_idx[word] for word in ["<START>", "consult", "your", "doctor", "before", "mixing", "medications", "<END>"]],
    [word_to_idx[word] for word in ["<START>", "Ab", "and", "Ba", "can", "be", "taken", "together", "<END>"]],
]

# Simulate a noisy version of the text by shuffling the words
def add_noise(sequence):
    noisy_sequence = sequence.copy()
    mid_part = noisy_sequence[1:-1]
    random.shuffle(mid_part)
    return [noisy_sequence[0]] + mid_part + [noisy_sequence[-1]]

# Training loop
for step in range(num_steps):
    for sequence in training_data:
        noisy_sequence = torch.tensor([add_noise(sequence)], dtype=torch.long)
        original_sequence = torch.tensor([sequence[1:]], dtype=torch.long)  # Target is the sequence without the start token

        # Forward pass
        output = model(noisy_sequence)

        # Compute loss
        loss = criterion(output.view(-1, vocab_size), original_sequence.view(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}, Loss: {loss.item()}")

# Inference (Denoising Process)
def generate_text(noisy_sequence, num_iterations=5):
    noisy_sequence = torch.tensor([noisy_sequence], dtype=torch.long)
    for _ in range(num_iterations):
        with torch.no_grad():
            prediction = model(noisy_sequence)
            noisy_sequence = prediction.argmax(dim=-1)
    return noisy_sequence[0].tolist()

# Example noisy input
noisy_input = add_noise([word_to_idx[word] for word in ["<START>", "you", "should", "take", "Ab", "instead", "of", "Ba", "<END>"]])

# Generate denoised text
generated_sequence = generate_text(noisy_input)
generated_text = " ".join([idx_to_word[idx] for idx in generated_sequence])

print("Generated Text:", generated_text)
